In [4]:
from datetime import datetime
import tensorflow as tf
import importlib
import numpy as np
import pandas as pd


In [5]:
# import config 
import configs.config
importlib.reload(configs.config)
from configs.config import CFG
from utils import config
config = config.Config.from_json(CFG)

## Create Dataset

In [6]:
# create Train Series and Dataset 
import dataloader.dataloader
importlib.reload(dataloader.dataloader)
from dataloader.dataloader import Train_Series,Dataset
try: 
    del series,ds
except:
    pass
from dataloader.dataloader import Test_Series
try: 
    del test_series
except:
    pass

series = Train_Series(config.data,config.paths)
series.createSeries()
test_series = Test_Series(config.data,config.paths)
test_series.createSeries()
ds = Dataset(series,test_series)
ds.split_dataset()

X_train = ds.X_train
y_train = ds.y_train
X_test = ds.X_test
y_test = ds.y_test

print(X_train.shape)
print(X_test.shape)


(10863, 160, 2)
(3, 160, 2)


In [7]:
ds.mean

array([-7.08130408,  0.02767148])

In [8]:
ds.epsilon

1e-15

In [9]:
ds.std

array([42.23753344,  0.07083532])

## Save Datset to Files 

In [10]:
# import pickle
# # Include the epoch in the file name (uses `str.format`)
# ds_path = "./model/saved_ds/ds.obj"

# with  open(ds_path,'wb') as file:
#     exampleObj = pickle.dump(ds,file)


## Call model and compile

In [13]:
from model.model1D import Unet,Unet_Model
import model.model1D
importlib.reload(model.model1D)
try: 
    del unet
except:
    pass

# def MySCCE_Loss(y_true,y_pred):
#     loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False, ignore_class= 0)
#     return loss 


unet = Unet(n_classes=3,n_filters=32)
unet.compile(optimizer='adam',
              #loss = MySCCE_Loss,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['sparse_categorical_accuracy'],
              run_eagerly=True)
unet.model((160,2)).summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 160, 2)]     0           []                               
                                                                                                  
 downsampling_block_5 (Downsamp  ((None, 80, 32),    3328        ['input_2[0][0]']                
 lingBlock)                      (None, 160, 32))                                                 
                                                                                                  
 downsampling_block_6 (Downsamp  ((None, 40, 64),    18560       ['downsampling_block_5[0][0]']   
 lingBlock)                      (None, 80, 64))                                                  
                                                                                            

## Train Model and save weights and model to files

In [16]:
from datetime import datetime
#train model
history = unet.fit(X_train,y_train,epochs =1,batch_size = 256)




43/43 [==============================] - 49s 1s/step - loss: 0.2075 - sparse_categorical_accuracy: 0.9292


In [17]:
now = str(datetime.now().date())
# Include the epoch in the file name (uses `str.format`)
checkpoint_path = "model/weights/cp-{epoch:04d}_{now}.ckpt"
# save weights 
unet.save_weights(checkpoint_path.format(epoch=100,now=now))
#save model
models_path = "model/saved_models/unet_{version}_{now}"
unet.save(models_path.format(version = "v1", now = now))

INFO:tensorflow:Assets written to: model/saved_models/unet_v1_2023-11-08\assets


INFO:tensorflow:Assets written to: model/saved_models/unet_v1_2023-11-08\assets


In [ ]:
# # Include the epoch in the file name (uses `str.format`)
# checkpoint_path = "model/weights/cp-{epoch:04d}.ckpt"

# # Loads the weights
# unet.load_weights(checkpoint_path.format(epoch =100))

# # Re-evaluate the model
# # loss, acc = unet.evaluate(test_images, test_labels, verbose=2)
# # print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

In [ ]:
# def create_events(serie,y_pred :np.array):
#     """create df_events 
#     input:
#     y_pred - score  (predicited vaues )"""
#     def detectChange(last_val,current_val):
#         if last_val == -1 or current_val == -1:
#             return False

#         return last_val !=current_val

#     # create empty dataframe
#     df_events = pd.DataFrame(columns=["series_id","step","event","score"])
#     # unpad y_pred
#     if serie.slice_pads is None:
#         y_pred_unpadded = y_pred
#     else:
#         y_pred_unpadded = serie._unpad(y_pred,serie.slice_pads)
#     # create step seg list
#     #         - events - segmentation mask
#     #         -  score   - predicited vaues for chosen event
#     events = np.argmax(y_pred_unpadded,axis = -1,keepdims=True)
#     scores = np.max(y_pred_unpadded,axis=-1,keepdims=True)      


#     for slices_num in range(events.shape[0]):
#         slice = events[slices_num]
#         score = scores[slices_num]
#         for i in range(events.shape[1]):
#             event_val = slice[i][0]
#             event_score = score[i][0]

#             if i == 0:
#                 # do not detect anything during first step
#                 continue
#             elif not detectChange(slice[i-1],event_val):
#                 continue

#             df_events.loc[len(df_events.index)] = [serie.serie_id,i,event_val,event_score]

#     # decode events 
#     decoded_list = {v: k for k, v in serie.__class__.decode_list.items()}
#     df_events["event"] = df_events["event"].map(decoded_list)  
#     # df_events = serie.decode_events(df_events)
#     # save as serie events 
#     return df_events 

In [ ]:
# def predict(model,serie):
#     x = serie.get_correct_slices()
#     x = x[...,:-1]
#     try:
#         y_pred = model.predict(x)
#     except ValueError:
#         print(x.shape)
#         print(x)
#         print(y_pred)
        
#     # print(y_pred.shape)
#     #print(y_pred)
#     serie_events = create_events(serie, y_pred)   
#     return serie_events

In [20]:
from dataloader.dataloader import Serie

In [21]:
def predict(model,serie):
    x = serie.get_correct_slices()
    x = x[...,0:2]
    print(x)
    print(x.shape)
    try:
        y_pred = model.predict(x)
    except ValueError:
        print(x.shape)
        print(x)

        
    # print(y_pred.shape)
    #print(y_pred)
    serie_events , raw_prediction = Serie.create_events(serie,y_pred)   
    return serie_events, raw_prediction

In [ ]:
# serie= series.series["04f547b8017d"].get_correct_slices()
# serie[serie[:,:,-1] == 2.0].shape

(151392, 3)

## Validation with train set

In [ ]:
# ## 
# raw_predictions = {}
# series_pred = {}
# y_pred = None
# pred_start = False
# for serie_id, serie in series.series.items():
#     # print(serie_id)
#     serie_events, raw_predictions[serie_id] = predict(unet,serie)

#     if pred_start == False:
#         y_pred = serie_events
#         pred_start = True
#     else:
#         y_pred = pd.concat([y_pred, serie_events])
# y_pred

[[[ 4.01249e+01  0.00000e+00]
  [ 4.03475e+01  0.00000e+00]
  [ 4.03475e+01  0.00000e+00]
  ...
  [ 6.98528e+01  0.00000e+00]
  [ 6.98528e+01  0.00000e+00]
  [ 6.98528e+01  0.00000e+00]]

 [[ 6.39758e+01  0.00000e+00]
  [ 6.39758e+01  0.00000e+00]
  [ 6.39758e+01  0.00000e+00]
  ...
  [-2.92967e+01  4.71000e-02]
  [-5.59164e+01  3.54000e-02]
  [-4.45055e+01  7.83000e-02]]

 [[ 1.94255e+01  0.00000e+00]
  [ 1.94255e+01  0.00000e+00]
  [ 1.94255e+01  0.00000e+00]
  ...
  [ 7.15540e+00  0.00000e+00]
  [ 7.15540e+00  0.00000e+00]
  [ 7.15540e+00  0.00000e+00]]

 ...

 [[-8.54780e+01  0.00000e+00]
  [-8.54780e+01  0.00000e+00]
  [-8.54780e+01  0.00000e+00]
  ...
  [ 2.91774e+01  2.00000e-04]
  [-6.20890e+00  2.28000e-02]
  [-1.83730e+01  7.72000e-02]]

 [[-3.01340e+00  2.64000e-02]
  [-2.04410e+00  1.30000e-02]
  [-1.17010e+00  6.80000e-03]
  ...
  [ 6.26755e+01  2.90000e-03]
  [ 6.26755e+01  2.90000e-03]
  [ 6.99417e+01  2.78000e-02]]

 [[ 1.72105e+01  9.00000e-04]
  [ 1.89250e+01  0.00000

,series_id,step,event,score
0,03d92c9f6f8a,159,onset,15.114036
1,03d92c9f6f8a,158,onset,12.920636
2,03d92c9f6f8a,105,onset,1.440894
3,03d92c9f6f8a,28,onset,1.782987
4,03d92c9f6f8a,185,onset,3.064545
...,...,...,...,...
24,7fd4284b7ee8,4770,onset,1.152995
25,7fd4284b7ee8,3813,onset,3.641382
26,7fd4284b7ee8,5056,onset,9.056438
27,7fd4284b7ee8,2211,onset,4.351101


### Check Test Set

In [22]:
raw_predictions = {}
series_pred = {}
y_pred = None
pred_start = False
for serie_id, serie in test_series.series.items():
    serie_events, raw_predictions[serie_id] = predict(unet,serie)
    print(serie_events)
    if pred_start == False:
        y_pred = serie_events
        pred_start = True
    else:
        y_pred = pd.concat([y_pred, serie_events])



[[[ 2.63670e+00  2.17000e-02]
  [ 2.63680e+00  2.15000e-02]
  [ 2.63700e+00  2.16000e-02]
  [ 2.63680e+00  2.13000e-02]
  [ 2.63680e+00  2.15000e-02]
  [ 2.63670e+00  2.17000e-02]
  [ 2.63670e+00  2.17000e-02]
  [ 2.63670e+00  2.18000e-02]
  [ 2.79800e+00  2.23000e-02]
  [ 3.08470e+00  2.17000e-02]
  [ 2.78020e+00  2.29000e-02]
  [ 2.52050e+00  2.16000e-02]
  [ 2.41290e+00  2.18000e-02]
  [ 2.41290e+00  2.14000e-02]
  [ 2.41290e+00  2.14000e-02]
  [ 2.41290e+00  2.14000e-02]
  [ 2.41290e+00  2.15000e-02]
  [ 2.41320e+00  2.17000e-02]
  [ 4.98744e+01  3.95000e-02]
  [ 5.48498e+01  1.66000e-02]
  [ 4.40993e+01  1.70000e-02]
  [ 6.73813e+01  1.02000e-02]
  [ 6.84605e+01  2.38000e-02]
  [ 3.77219e+01  7.00000e-03]
  [ 3.00025e+01  8.20000e-03]
  [ 2.78679e+01  1.19000e-02]
  [ 2.85856e+01  1.15000e-02]
  [-6.60690e+00  6.91000e-02]
  [-7.99704e+01  1.32000e-02]
  [-7.99883e+01  1.36000e-02]
  [-7.99987e+01  1.34000e-02]
  [-7.99780e+01  1.35000e-02]
  [-7.99694e+01  1.33000e-02]
  [-7.9968

In [23]:

y_pred

,series_id,step,event,score
0,038441c925bb,27,onset,151.022415
0,03d92c9f6f8a,27,onset,183.628708


In [33]:
raw_predictions["038441c925bb"]

array([[[  0.        ,  14.56332397,   8.4055624 ],
        [  0.        ,  17.51624298,  11.70478058],
        [  0.        ,  29.46958351,  22.11977386],
        [  0.        ,  28.08721924,  22.17699623],
        [  0.        ,  31.49054909,  24.64376068],
        [  0.        ,  30.65968704,  23.63072395],
        [  0.        ,  30.37230682,  23.72773933],
        [  0.        ,  32.34988022,  26.08835983],
        [  0.        ,  26.708601  ,  20.61184883],
        [  0.        ,  31.0080452 ,  23.75686455],
        [  0.        ,  30.25430107,  23.06399918],
        [  0.        ,  37.65042496,  28.89154053],
        [  0.        ,  40.32673645,  28.96580315],
        [  0.        ,  44.97067642,  37.97266388],
        [  0.        ,  51.28723145,  36.79949951],
        [  0.        ,  68.99363708,  59.63228989],
        [  0.        ,  87.71109009,  75.30039215],
        [  0.        , 104.96044159, 100.70440674],
        [  0.        , 119.95076752, 100.5400238 ],
        [  0

In [43]:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x)
    return e_x / np.expand_dims(e_x.sum(axis=-1),axis = -1) # only difference

pred = softmax(raw_predictions["038441c925bb"])


array([[[4.72400263e-07, 9.97887014e-01, 2.11251353e-03],
        [2.46317005e-08, 9.97015857e-01, 2.98411814e-03],
        [1.58943751e-13, 9.99357698e-01, 6.42301875e-04],
        [6.31974626e-13, 9.97295751e-01, 2.70424932e-03],
        [2.10555674e-14, 9.98938265e-01, 1.06173522e-03],
        [4.83373182e-14, 9.99114934e-01, 8.85065817e-04],
        [6.44035880e-14, 9.98700620e-01, 1.29938044e-03],
        [8.90835547e-15, 9.98095293e-01, 1.90470734e-03],
        [2.50972323e-12, 9.97754888e-01, 2.24511215e-03],
        [3.41247222e-14, 9.99291166e-01, 7.08833616e-04],
        [7.25098015e-14, 9.99246706e-01, 7.53293620e-04],
        [4.45204961e-17, 9.99842965e-01, 1.57035058e-04],
        [3.06419099e-18, 9.99988359e-01, 1.16413756e-05],
        [2.94500984e-20, 9.99087138e-01, 9.12862016e-04],
        [5.32400150e-23, 9.99999489e-01, 5.10572882e-07],
        [1.08744371e-30, 9.99914023e-01, 8.59767902e-05],
        [8.08268779e-39, 9.99995925e-01, 4.07474654e-06],
        [2.571

In [37]:
dense = raw_predictions["038441c925bb"]
outputs = tf.keras.layers.Dense(units=3,activation='softmax')(dense)
outputs

<tf.Tensor: shape=(1, 150, 3), dtype=float32, numpy=
array([[[1.0000000e+00, 1.6133633e-15, 4.5264018e-12],
        [1.0000000e+00, 2.1317039e-19, 4.3766019e-15],
        [1.0000000e+00, 1.7712409e-33, 5.6893943e-26],
        [1.0000000e+00, 1.5161625e-32, 2.8108958e-25],
        [1.0000000e+00, 2.7975512e-36, 3.7386576e-28],
        [1.0000000e+00, 3.8454943e-35, 2.8767061e-27],
        [1.0000000e+00, 5.3932674e-35, 3.6728004e-27],
        [1.0000000e+00, 1.1238845e-37, 3.0088996e-29],
        [1.0000000e+00, 1.0125831e-30, 7.3688680e-24],
        [1.0000000e+00, 1.8725721e-35, 1.6637924e-27],
        [1.0000000e+00, 1.5143921e-34, 8.3893108e-27],
        [1.0000000e+00, 0.0000000e+00, 2.9122952e-33],
        [1.0000000e+00, 0.0000000e+00, 1.0918582e-34],
        [1.0000000e+00, 0.0000000e+00, 0.0000000e+00],
        [1.0000000e+00, 0.0000000e+00, 0.0000000e+00],
        [1.0000000e+00, 0.0000000e+00, 0.0000000e+00],
        [1.0000000e+00, 0.0000000e+00, 0.0000000e+00],
        [1.0

In [38]:
np.sum(raw_predictions["038441c925bb"],axis = -1)

array([[ 22.96888638,  29.22102356,  51.58935738,  50.26421547,
         56.13430977,  54.290411  ,  54.10004616,  58.43824005,
         47.32044983,  54.76490974,  53.31830025,  66.54196548,
         69.2925396 ,  82.9433403 ,  88.08673096, 128.62592697,
        163.01148224, 205.66484833, 220.49079132, 250.90155029,
        294.86431885, 260.89287567, 283.09571838, 256.43621826,
        267.00823975, 274.54655457, 275.24249268, 296.14822388,
        266.04075623, 295.49299622, 280.4254303 , 317.42881775,
        270.20022583, 294.24475098, 271.99319458, 308.67723083,
        284.62625122, 300.2885437 , 282.01359558, 310.28024292,
        292.05647278, 300.9226532 , 279.68334961, 304.65478516,
        288.02030945, 295.84762573, 278.25271606, 301.02960205,
        286.43084717, 294.57319641, 277.71211243, 299.62913513,
        285.4216156 , 294.07420349, 277.91604614, 300.08364868,
        285.15562439, 293.99359131, 276.00474548, 300.8447113 ,
        286.09005737, 295.98118591, 278.

In [ ]:
y_pred.to_csv('submission.csv', index=True,index_label = "row_id")

In [ ]:
# from matplotlib import pyplot as plt
# plt.plot(history.history["accuracy"])

# def accuracy(real,pred):
#     comp = real == pred

#     return np.sum(comp)/(real.shape[0]*real.shape[1])

# arg = np.expand_dims(np.argmax(y_dev_pred,axis = -1),axis = -1)
# print("train accuracy : " + str(accuracy(y_dev,arg)))



In [ ]:
# from matplotlib import pyplot as plt
# plt.plot(history.history["accuracy"])

# def accuracy(real,pred):
#     comp = real == pred
#     return np.sum(comp)/(comp.shape[0]*comp.shape[1])


# y_train_pred = unet.predict(X_train)
# print("train accuracy : " + str(accuracy(y_train,y_train_pred)))


# y_test_pred = unet.predict(X_test)
# print("test accuracy : " +  str(accuracy(y_test,y_test_pred)))

In [ ]:
# from IPython.display import FileLink
# import pandas as pd

# # import os
# # os.chdir(r'kaggle/working')'
# filename = 'train_' +'.csv'
# pd.DataFrame(y_pred).reset_index().to_csv(filename, index=True,index_label = "row_id")

# FileLink(filename)



d:\DEV\Projekty\PythonProjects\Kaggle\Detect-sleep-state\train_.csv